# Load Libraries

In [1]:
%run "../../Libraries/Display.ipynb"
%run "../../Libraries/Standard.ipynb"

# Read Data

In [2]:
sage    = pd.read_csv("../../Data/ITPS/sage.csv")
manager = pd.read_csv("../../Data/ITPS/manager.csv")

In [3]:
months = ['Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
       'Oct-16', 'Nov-16', 'Dec-16', 'Jan-17', 'Feb-17', 'Mar-17', 'Apr-17',
       'May-17', 'Jun-17', 'Jul-17', 'Aug-17', 'Sep-17', 'Oct-17', 'Nov-17',
       'Dec-17', 'Jan-18', 'Feb-18', 'Mar-18', 'Apr-18', 'May-18', 'Jun-18',
       'Jul-18', 'Aug-18', 'Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19',
       'Feb-19', 'Mar-19', 'Apr-19', 'May-19']

In [4]:
manager = manager.drop(['31/5/16', '30/06/2016', '31/07/2016', '31/08/2016', '30/09/2016',
       '31/10/2016', '30/11/2016', '31/12/2016', '31/01/2017', '28/02/2017',
       '31/03/2017', '30/04/2017', '31/05/2017', '30/06/2017', '31/07/2017',
       '31/08/2017', '30/09/2017', '31/10/2017', '30/11/2017', '31/12/2017',
       '31/01/2018', '28/02/2018', '31/03/2018', '30/04/2018', '31/05/2018',
       '30/06/2018', '31/07/2018', '31/08/2018', '30/09/2018', '31/10/2018',
       '30/11/2018', '31/12/2018', '31/01/2019', '28/02/2019', '31/03/2019',
       '30/04/2019', '31/05/2019'], axis=1)

In [5]:
for month in months: 
    manager[month] = manager[month].astype(str)
    manager[month] = manager[month].apply(lambda x: x.replace(",", ""))
    manager[month] = manager[month].apply(lambda x: x.strip())
    manager[month] = manager[month].astype(float)

In [6]:
sage['Inv No']   = sage['NLPostedNominalTrans.Reference']

sage['Nom Code'] = sage['NLNominalAccounts.AccountNumber']

# Change manually the date - bug
#sage['Month'] = pd.to_datetime(sage['NLPostedNominalTrans.TransactionDate'])

sage = sage.drop(['NLPostedNominalTrans.Reference', 'NLNominalAccounts.AccountNumber', 'NLPostedNominalTrans.TransactionDate'], axis=1)

In [7]:
sage.sort_index(inplace=True)

manager.sort_index(inplace=True)

# Dates Format

In [8]:
# Change manually the date - bug - check above!
# sage['Month'] = sage['Month'].apply(lambda x: x.strftime("%B")[:3] + str("-") + x.strftime("%Y")[2:4])

# sage['Month'] = sage['Month'].astype(str)

# Fill Empty Invoices

In [9]:
for index, row in sage[ pd.isna(sage['Inv No']) ].iterrows():
    sage.loc[index, 'Inv No'] = 'sage_' + str(row['NLPostedNominalTrans.UniqueReferenceNumber']) + "_" + str(index)
    
for index, row in manager[ pd.isna(manager['Inv No']) ].iterrows():
    manager.loc[index, 'Inv No'] = 'manager_' + str(index)

In [10]:
# Filter
sage = sage[sage['Nom Code'] == 2010]

# Make 'Inv No' UPPER

In [11]:
sage['Inv No'] = sage['Inv No'].apply(lambda x: x.upper())

manager['Inv No'] = manager['Inv No'].apply(lambda x: x.upper())

# Find Indexes

In [12]:
sage_index = sage['Inv No'].unique()

manager_index = manager['Inv No'].unique()

common_index = list(set(sage_index).intersection(manager_index))

# Find Not in (Manger and Sage) & Group Based On 'Inv No'

In [13]:
not_in_sage = manager[ ~manager['Inv No'].isin(common_index) ]

not_in_manager = sage[ ~sage['Inv No'].isin(common_index) ]

group_not_in_sage = not_in_sage.groupby(['Inv No']).sum()

group_not_in_manager = not_in_manager.groupby(['Inv No', 'Month']).sum()

In [14]:
group_not_in_sage.to_csv("../../Data/ITPS/Generated Files/group_not_in_sage.csv")

group_not_in_manager.to_csv("../../Data/ITPS/Generated Files/group_not_in_manager.csv")

group_not_in_sage = pd.read_csv("../../Data/ITPS/Generated Files/group_not_in_sage.csv")

group_not_in_manager = pd.read_csv("../../Data/ITPS/Generated Files/group_not_in_manager.csv")

# Find Common & Group Based On 'Inv No'

In [15]:
common_manager = manager[ manager['Inv No'].isin(common_index) ]

common_sage = sage[ sage['Inv No'].isin(common_index) ]

group_common_manager = common_manager.groupby(['Inv No']).sum()

group_common_sage = common_sage.groupby(['Inv No', 'Month']).sum()

In [16]:
group_common_manager.to_csv("../../Data/ITPS/Generated Files/group_common_manager.csv")

group_common_sage.to_csv("../../Data/ITPS/Generated Files/group_common_sage.csv")

group_common_manager = pd.read_csv("../../Data/ITPS/Generated Files/group_common_manager.csv")

group_common_sage = pd.read_csv("../../Data/ITPS/Generated Files/group_common_sage.csv")

In [17]:
merge_common = pd.merge( group_common_sage, group_common_manager, on=['Inv No'] )

merge_common.to_csv("../../Data/ITPS/Generated Files/merge_common.csv")

# Find Differences

In [18]:
merge_common = pd.read_csv("../../Data/ITPS/Generated Files/merge_common.csv")

merge_common['Common_Difference'] = np.nan

In [19]:
invoice_merge_index = merge_common['Inv No'].unique()

In [20]:
i = 0

for index, row in merge_common.iterrows():
    merge_common.iloc[i, -1] = -1*row['NLPostedNominalTrans.GoodsValueInBaseCurrency'] - row[row['Month']]            
    i = i + 1

In [22]:
merge_common_index = merge_common.shape[0]

In [23]:
for invoice in invoice_merge_index:
    sage_months = merge_common[merge_common['Inv No'] == invoice]['Month']
    
    exclude_months = list(set(months) - set(sage_months))
    
    row = merge_common[merge_common['Inv No'] == invoice].head(1)
    
    for month in exclude_months:
        row['Month'] = month        
        row['Common_Difference'] = -1*row[row['Month']]
        merge_common = merge_common.append(row, ignore_index=True)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800


In [24]:
merge_common.loc[merge_common_index:, 'NLPostedNominalTrans.GoodsValueInBaseCurrency'] = 0

In [25]:
merge_common.to_csv("../../Data/ITPS/Generated Files/merge_common_final.csv")

In [26]:
temp = []
for month in months:
    difference = round(sage[ sage['Month'] == month ].sum()['NLPostedNominalTrans.GoodsValueInBaseCurrency']*-1 + 
                       group_not_in_manager[group_not_in_manager['Month'] == month].sum()['NLPostedNominalTrans.GoodsValueInBaseCurrency'] + 
                       group_not_in_sage[month].sum() - merge_common[merge_common['Month'] == month].sum()['Common_Difference'] - manager[month].sum(),2)

    temp.append((month, 
              sage[ sage['Month'] == month ].sum()['NLPostedNominalTrans.GoodsValueInBaseCurrency'], 
              manager[month].sum(),
              merge_common[merge_common['Month'] == month].sum()['Common_Difference'],
              group_not_in_manager[group_not_in_manager['Month'] == month].sum()['NLPostedNominalTrans.GoodsValueInBaseCurrency'],
              group_not_in_sage[month].sum(), 
              difference
             ))

sum_up = pd.DataFrame(temp, columns=('Month', 'Sage', 'Manager', 'Common_Difference', 'Not_in_Sage', 'Not_in_Manager', 'Difference'))

In [27]:
sum_up.to_csv("../../Data/ITPS/Generated Files/sum_all.csv")